In [121]:
from environments.medevac import MedevacDispatchingEnvironment
from algorithms.ddqn import DQN
from gymnasium.wrappers import FlattenObservation
import numpy as np
import scipy
import torch

In [122]:
NUM_ZONES = 12
ALPHA = 0.25
policy_name = "zones12_alpha025"
policy_net_path = Path("experiment_results") / policy_name / "superlative.pth"

In [123]:
env = MedevacDispatchingEnvironment(
    map_config_file="environments\\configuration\\afghanistan\\v2\\map.csv",
    MTF_config_file="environments\\configuration\\afghanistan\\v2\\MTFs.csv", 
    staging_area_config_file="environments\\configuration\\afghanistan\\v2\\staging_areas.csv",
    casualty_cluster_center_config_file="environments\\configuration\\afghanistan\\v2\\casualty_cluster_centers.csv",
    intensity_function_config_file="environments\\configuration\\afghanistan\\v2\\intensity_function_ranges_1.csv",
    num_zones=NUM_ZONES,
    forecast_recency_bias=ALPHA,
    verbose=False
    )

env = FlattenObservation(env)

state_dim = env.observation_space.shape[0]
print(f"State space dimensionality: {state_dim}")

n_actions = env.action_space.n
print(f"Number of actions: {n_actions}")

State space dimensionality: 338
Number of actions: 33


In [124]:
n_neurons = 256
policy_net = DQN(state_dim, n_actions, n_neurons)
from pathlib import Path
policy_net_state_dict = torch.load(policy_net_path, weights_only=True)
policy_net.load_state_dict(policy_net_state_dict)
policy_net.eval()

DQN(
  (network): Sequential(
    (0): Linear(in_features=338, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=33, bias=True)
  )
)

In [125]:
from scipy.stats import t

def confinterval(data, alpha=0.05):
    n = np.size(data) # number of data points
    if n <= 1:
        raise ValueError("At least 2 data points are required to calculate a confidence interval.")
    sample_std = np.std(data, ddof=1) # sample standard deviation (ddof=1)
    se = sample_std / np.sqrt(n) # standard error
    t_score = t.ppf(1 - alpha / 2, n - 1) 
    mean = np.mean(data)
    halfwidth = t_score * se
    return mean, halfwidth

In [126]:
def evaluate_episode(policy_net, rep, seed_mult, offset, is_constrained=True): 
        terminated = False
        truncated = False
        cumulative_reward = 0
        state, info = env.reset(seed=int(seed_mult * 1000 + rep + offset))

        with torch.no_grad():
            while not(terminated or truncated):
                
                state_action_values = policy_net(
                    torch.as_tensor(state, dtype=torch.float32, device="cpu").unsqueeze(0)
                    ).squeeze(0)

                if is_constrained:
                    mask = torch.tensor(info['mask'], dtype=torch.bool, device="cpu")
                    state_action_values[~mask] = float('-inf')
                
                action = state_action_values.argmax().item()

                state, reward, terminated, truncated, info = env.step(action)
                cumulative_reward += reward        

        return cumulative_reward

In [131]:
import pandas as pd

def get_service_metrics_df(metrics):
    data = []
    # Flatten the metrics into rows
    for mtf, precedences in metrics["avg_service_times"].items():
        for precedence in precedences.keys():
            data.append({
                "MTF": mtf,
                "Precedence": precedence.to_string(),
                "avg_service_times": metrics["avg_service_times"][mtf][precedence],
                "total_num_requests_serviced": metrics["total_num_requests_serviced"][mtf][precedence],
                "num_requests_serviced_on_time": metrics["num_requests_serviced_on_time"][mtf][precedence],
                "percent_requests_serviced_on_time": metrics["percent_requests_serviced_on_time"][mtf][precedence],
            })
    return pd.DataFrame(data)

def evaluate(policy_name, policy_net, num_reps, seed_mult, offset): 
    # Initialize test data structures
    test_data = np.zeros((num_reps))
    all_metrics = []
    zone_metrics = []
    staging_area_metrics = []

    # Run num_reps replications per test
    for rep in range(num_reps):
        Gtest = evaluate_episode(policy_net, rep, seed_mult, offset)
        test_data[rep] = Gtest

        # Get request manager service metrics
        metrics = env.unwrapped.request_manager.get_service_metrics()
        metrics_df = get_service_metrics_df(metrics)
        metrics_df['rep'] = rep
        metrics_df['MTF'] = metrics_df['MTF'].astype(str)  # Convert MTF to string
        all_metrics.append(metrics_df)

        # Collect staging area metrics per staging area
        for staging_area in env.unwrapped.staging_areas:
            sa_metrics = staging_area.get_service_metrics()
            staging_area_metrics.append({
                "Staging Area": str(staging_area),  # Convert staging area object to string
                "num_helicopters_serviced": sa_metrics["num_helicopters_serviced"]
            })

        # Get service metrics for each zone
        for zone in env.unwrapped.zones:
            zone_metrics_df = get_service_metrics_df(zone.get_service_metrics())
            zone_metrics_df['rep'] = rep
            zone_metrics_df['Zone'] = str(zone)  # Convert Zone object to string
            zone_metrics.append(zone_metrics_df)

    # Concatenate all collected metrics into DataFrames
    all_metrics_df = pd.concat(all_metrics, ignore_index=True)
    zone_metrics_df = pd.concat(zone_metrics, ignore_index=True)
    staging_area_metrics_df = pd.DataFrame(staging_area_metrics)

    # Compute weighted average function
    def weighted_avg(series, weights):
        return (series * weights).sum() / weights.sum() if weights.sum() > 0 else 0

    # Compute weighted averages for request manager service metrics
    weighted_avg_df = all_metrics_df.groupby(['MTF', 'Precedence']).agg(
        avg_service_times=('avg_service_times', lambda x: weighted_avg(x, all_metrics_df.loc[x.index, 'total_num_requests_serviced'])),
        total_num_requests_serviced=('total_num_requests_serviced', 'sum'),
        num_requests_serviced_on_time=('num_requests_serviced_on_time', 'sum'),
        percent_requests_serviced_on_time=('num_requests_serviced_on_time', 
            lambda x: 100 * x.sum() / all_metrics_df.loc[x.index, 'total_num_requests_serviced'].sum() 
            if all_metrics_df.loc[x.index, 'total_num_requests_serviced'].sum() > 0 else 0)
    ).reset_index()

    # Compute weighted averages for zone service metrics
    weighted_zone_avg_df = zone_metrics_df.groupby(['Zone', 'Precedence']).agg(
        avg_service_times=('avg_service_times', lambda x: weighted_avg(x, zone_metrics_df.loc[x.index, 'total_num_requests_serviced'])),
        total_num_requests_serviced=('total_num_requests_serviced', 'sum'),
        num_requests_serviced_on_time=('num_requests_serviced_on_time', 'sum'),
        percent_requests_serviced_on_time=('num_requests_serviced_on_time', 
            lambda x: 100 * x.sum() / zone_metrics_df.loc[x.index, 'total_num_requests_serviced'].sum() 
            if zone_metrics_df.loc[x.index, 'total_num_requests_serviced'].sum() > 0 else 0)
    ).reset_index()

    # Aggregate helicopter servicing counts for each staging area
    total_staging_area_metrics_df = staging_area_metrics_df.groupby('Staging Area', as_index=False).sum()

    # Save results to CSV
    weighted_avg_df.to_csv(f'{policy_name}_weighted_avg_service_metrics.csv', index=False)
    weighted_zone_avg_df.to_csv(f'{policy_name}_weighted_zone_metrics.csv', index=False)
    total_staging_area_metrics_df.to_csv(f'{policy_name}_staging_area_metrics.csv', index=False)

    mean, hw = confinterval(test_data)
    return mean, hw, weighted_avg_df



In [132]:
num_reps = 60
seed_mult = 2
offset = 0

In [133]:
mean, hw, weighted_avg_df = evaluate(policy_name,policy_net, num_reps, seed_mult, offset)
print(f"Mean: {mean}, Halfwidth: {hw}")
print(weighted_avg_df)

Mean: 73.48338865679224, Halfwidth: 1.4759111332658597
    MTF Precedence  avg_service_times  total_num_requests_serviced  \
0  MTF1   Priority           1.018242                          446   
1  MTF1    Routine           1.201658                          414   
2  MTF1     Urgent           2.505707                         2842   
3  MTF2   Priority           1.412696                         5830   
4  MTF2    Routine           1.878680                         2950   
5  MTF2     Urgent           0.000000                            0   

   num_requests_serviced_on_time  percent_requests_serviced_on_time  
0                            446                         100.000000  
1                            414                         100.000000  
2                            716                          25.193526  
3                           5748                          98.593482  
4                           2950                         100.000000  
5                              0  